In [ ]:
!pip -q install langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.4/648.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.150
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity, tqdm
Required-by: 


# Using LangChain with some Tools Wikipedia DuckDuckGo PythonRepl

### Setting up wikipedia

In [ ]:
!pip -q install wikipedia

  Preparing metadata (setup.py) ... done


In [ ]:
from langchain.utilities import WikipediaAPIWrapper

In [ ]:
wikipedia = WikipediaAPIWrapper()

In [ ]:
wikipedia.run('Langchain')

'Page: LangChain\nSummary: LangChain is a software development framework designed to simplify the creation of applications using large language models (LLMs).\n\n\n\nPage: Software development\nSummary: Software development is the process of conceiving, specifying, designing, programming, documenting, testing, and bug fixing involved in creating and maintaining applications, frameworks, or other software components. Software development involves writing and maintaining the source code, but in a broader sense, it includes all processes from the conception of the desired software through the final manifestation, typically in a planned and structured process often overlapping with software engineering. Software development also includes research, new development, prototyping, modification, reuse, re-engineering, maintenance, or any other activities that result in software products.\n\n\n\nPage: Software engineering\nSummary: Software engineering is an engineering-based approach to softwar

## REPL

In [ ]:
from langchain.utilities import PythonREPL

In [ ]:
python_repl = PythonREPL()

In [ ]:
python_repl.run("print(17*2)")

'34\n'

## Duck Duck Go

In [ ]:
!pip -q install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.8 MB/s eta 0:00:00


In [ ]:
from langchain.tools import DuckDuckGoSearchTool

In [ ]:
search = DuckDuckGoSearchTool()

In [ ]:
search.run("Tesla stock price?")

'Tesla, Inc. Common Stock (TSLA) Add to Watchlist Add to Portfolio Quotes Summary Real-Time After-Hours Key Data Bid Price and Ask Price The bid & ask refers to the price that an... Get the latest Tesla Inc (TSLA) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions. Discover historical prices for TSLA stock on Yahoo Finance. View daily, weekly or monthly format back to when Tesla, Inc. stock was issued. We reduce our Tesla fair value estimate to $215 per share from $225 to reflect higher near-term margin compression as the company is pursuing a volume-over-unit profit strategy in the near term... TSLA: Tesla Inc - Stock Price, Quote and News - CNBC'

## Putting them together

In [ ]:
from langchain import OpenAI

llm = OpenAI(temperature=0)

In [ ]:
tools = [
    Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    )
]

In [ ]:


wikipedia_tool = Tool(
    name='wikipedia',
    func= wikipedia.run,
    description="Useful for when you need to look up a topic, country or person on wikipedia"
)

duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)



In [ ]:
tools.append(duckduckgo_tool)
tools.append(wikipedia_tool)


## Using the agents

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [ ]:
zero_shot_agent.run("When was Barak Obama born?")



> Entering new AgentExecutor chain...
 I need to find out when Barak Obama was born.
Action: DuckDuckGo Search
Action Input: "Barack Obama birthdate"
Observation: Barack Obama, in full Barack Hussein Obama II, (born August 4, 1961, Honolulu, Hawaii, U.S.), 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08). Barack Hussein Obama II (/ b ə ˈ r ɑː k h uː ˈ s eɪ n oʊ ˈ b ɑː m ə / bə-RAHK hoo-SAYN oh-BAH-mə; born August 4, 1961) is an American former politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American president of the United States. Obama previously served as a U.S. senator representing ... Barack Hussein Obama, the 44th and first African-American President of the United States, served from 2008 until 2016. is celebrating his 60th Birthday in 2021. ... Birth d

'Barack Obama was born on August 4, 1961.'

In [ ]:
zero_shot_agent.run("What is 17*6?")



> Entering new AgentExecutor chain...
 I need to do a calculation
Action: python repl
Action Input: 17*6
Observation: 
Thought: I now know the answer
Final Answer: 102

> Finished chain.


'102'

In [ ]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

python repl: useful for when you need to use python to answer a question. You should input python code
DuckDuckGo Search: Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input.
wikipedia: Useful for when you need to look up a topic, country or person on wikipedia

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python repl, DuckDuckGo Search, wikipedia]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
zero_shot_agent.run("Tell me about LangChain")



> Entering new AgentExecutor chain...
 I need to find out what LangChain is
Action: DuckDuckGo Search
Action Input: "LangChain"
Observation: Welcome to LangChain. #. LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an API, but will also: Be data-aware: connect a language model to other sources of data. Be agentic: allow a language model to ... LangChain is an open-source Python library that enables anyone who can write code to build LLM-powered applications. The package provides a generic interface to many foundation models, enables prompt management, and acts as a central interface to other components like prompt templates, other LLMs, external data, and other tools via agents ... LangChain is a powerful framework designed to help developers build end-to-end applications using language models. It offers a suite of tools, components, and int

'LangChain is an open-source Python library that enables anyone who can write code to build LLM-powered applications. It offers a suite of tools, components, and interfaces that simplify the process of creating applications powered by large language models (LLMs) and chat models. LangChain makes it easy to manage interactions with language models and provides several specially created chains just for this purpose.'

In [ ]:
zero_shot_agent.run("Tell me about Singapore")



> Entering new AgentExecutor chain...
 Singapore is a country, so I should look it up on Wikipedia.
Action: wikipedia
Action Input: Singapore
Observation: Page: Singapore
Summary: Singapore ( (listen)), officially the Republic of Singapore, is an island country and city-state in maritime Southeast Asia. It is located about one degree of latitude (137 kilometres or 85 miles) north of the equator, off the southern tip of the Malay Peninsula, bordering the Strait of Malacca to the west, the Singapore Strait to the south, the South China Sea to the east, and the Straits of Johor to the north. The country's territory is composed of one main island, 63 satellite islands and islets, and one outlying islet; the combined area of these has increased by 25% since the country's independence as a result of extensive land reclamation projects. It has the third highest population density in the world, although there are numerous green and recreational spaces as a result of urban planning. With a mu

'Singapore is an island country and city-state in maritime Southeast Asia located about one degree of latitude north of the equator. It has the third highest population density in the world and is a highly developed country with the second-highest GDP per capita (PPP) in the world. It is a unitary parliamentary republic with a Westminster system of unicameral parliamentary government and its legal system is based on common law. The Singapore Army is the land service branch of the Singapore Armed Forces and is mostly made up of conscripts.'

In [ ]:
zero_shot_agent.run('what is the current price of btc')



> Entering new AgentExecutor chain...
 I need to find a source of information
Action: DuckDuckGo Search
Action Input: current price of btc
Observation: Bitcoin (BTC) Price, Real-time Quote & News - Google Finance Futures Dow Jones +0.066% +22.34 S&P 500 4,133.52 +0.090% Nasdaq +12.90 Russell 1,791.51 +1.81 VIX 16.77 -0.40 Home BTC / USDT... The live Bitcoin price today is $28,410.75 USD with a 24-hour trading volume of $21,867,658,528 USD. We update our BTC to USD price in real-time. Bitcoin is down 3.11% in the last 24 hours. The current CoinMarketCap ranking is #1, with a live market cap of $549,797,190,281 USD. Bitcoin (BTC) is a decentralized currency that eliminates the need for central authorities such as banks or governments by using a peer-to-peer internet network to confirm transactions directly between users. Price History $ 28,338.33 • 21:45 Vol 32 BTC 1D 1W 1M 1Y MAX Trade Bitcoin with the world's most popular crypto wallet. The current price of Bitcoin is $29,267.56 per 

'The current price of Bitcoin is $29,267.56 per BTC.'

In [ ]:
zero_shot_agent.run('Is 11 a prime number?')



> Entering new AgentExecutor chain...
 I need to find out if 11 is a prime number
Action: python repl
Action Input: 11 % 2 == 0
Observation: 
Thought: 11 is not divisible by 2, so it might be a prime number
Action: DuckDuckGo Search
Action Input: "Is 11 a prime number?"
Observation: 1The first 1000 prime numbers 2Lists of primes by type Toggle Lists of primes by type subsection 2.1Balanced primes 2.2Bell primes 2.3Chen primes 2.4Circular primes 2.5Cluster primes 2.6Cousin primes 2.7Cuban primes 2.8Cullen primes 2.9Dihedral primes 2.10Eisenstein primes without imaginary part 2.11Emirps 2.12Euclid primes It is possible to find out using mathematical methods whether a given integer is a prime number or not. For 11, the answer is: yes, 11 is a prime number because it has only two distinct divisors: 1 and itself (11). As a consequence, 11 is only a multiple of 1 and 11. What is a prime number? Since 11 is a prime number, 11 is also a deficient ... A prime number is divisible only by 1 and

'Yes, 11 is a prime number.'

In [ ]:
zero_shot_agent.run('Write a function to check if 11 a prime number and test it')



> Entering new AgentExecutor chain...
 I need to write a function to check if a number is prime
Action: python repl
Action Input: 
def is_prime(num):
    if num > 1:
        for i in range(2, num):
            if (num % i) == 0:
                return False
        else:
            return True
    else:
        return False
Observation: 
Thought: I need to test the function
Action: python repl
Action Input: 
print(is_prime(11))
Observation: True

Thought: I now know the final answer
Final Answer: 11 is a prime number.

> Finished chain.


'11 is a prime number.'